In [1]:
!pip install folium

In [6]:
!conda uninstall geopandas

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  removed specs: 
    - geopandas


The following packages will be REMOVED:

    geopandas: 0.4.1-py_0            
    osmnx:     0.10-py_1  conda-forge

Preparing transaction: done
Verifying transaction: done
Executing transaction: done


  Using cached https://files.pythonhosted.org/packages/69/77/98cbee7f94abe2c60013fd8751f2b2d6054d48078c61f9774cff0b341f43/osmnx-0.10-py2.py3-none-any.whl


In [51]:
!pip install geopandas
!pip install geocoder
!pip install rtree

In [ ]:
!pip force uninstall  osmnx

In [9]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import folium
import geocoder
import json
import urllib.request
import requests

from pandas.io.json import json_normalize
import rtree

In [49]:
import geopandas
import osmnx as osm

In [16]:
mestky_casty_url = 'http://opendata.iprpraha.cz/CUR/DTMP/TMMESTSKECASTI_P/S_JTSK/TMMESTSKECASTI_P.json'
results = requests.get(mestky_casty_url).json(encoding = "ISO 8859-2")
mestky_casty = json_normalize(results['features']) 
mestky_casty.head()

,geometry.coordinates,geometry.type,properties.DAT_VZNIK,properties.DAT_ZMENA,properties.ID,properties.ID_POSKYT,properties.KOD_MC,properties.KOD_MO,properties.KOD_SO,properties.NAZEV_1,properties.NAZEV_MC,properties.OBJECTID,properties.PLOCHA,properties.POSKYT,properties.STAV_ZMENA,properties.Shape_Area,properties.Shape_Length,properties.TID_TMMESTSKECASTI_P,type
0,"[[[-750495.6799868979, -1052524.009964496], [-...",Polygon,20180105160833,20190605145441,13,43,539635,51,132,Řeporyje,Praha-Řeporyje,1,9902539.83,HMP-IPR,U,9.902540e+06,24169.831485,13,Feature
1,"[[[-743677.7200093269, -1043880.2599875033], [...",Polygon,20180910110223,20180910113234,43,43,500054,19,19,Praha 1,Praha 1,2,5538443.86,HMP-IPR,U,5.538444e+06,12392.076909,43,Feature
2,"[[[-752038.7200328708, -1042852.1399567612], [...",Polygon,20181204155900,20181210155705,1,43,500178,60,60,Praha 6,Praha 6,3,41561204.34,HMP-IPR,U,4.156120e+07,49571.843399,1,Feature
3,"[[[-738597.8700456135, -1042441.1000244282], [...",Polygon,20170817145228,20170818091113,10,43,500216,94,94,Praha 9,Praha 9,4,13310394.12,HMP-IPR,U,1.331039e+07,20976.089531,10,Feature
4,"[[[-744523.739970047, -1038623.3699529357], [-...",Polygon,20181009145125,20190821104230,44,43,547271,60,60,Suchdol,Praha-Suchdol,5,5135601.75,HMP-IPR,U,5.135602e+06,13199.702891,44,Feature


In [22]:
result = []

result.append([
    v['properties']['ID'],
    v['properties']['NAZEV_MC'],
    v['geometry']['coordinates'],
    v['properties']['PLOCHA']] for v in results['features'])
    
df_prague = pd.DataFrame([item for result in result for item in result])
df_prague.columns = ['ID', 'Neigborouhood', 'Geometry', 'Area']

In [29]:
df_prague.sort_values('Neigborouhood', inplace = True)

In [32]:
dict_coordinates = {}
for index, row in df_prague.iterrows():
    g = geocoder.arcgis(row['Neigborouhood'])
    lat = g.json['lat']
    lng = g.json['lng']
    dict_coordinates[row['Neigborouhood']] = [lat, lng]
    
df_prague['latitude'] = 0.0
df_prague['longitude'] = 0.0
for k, v in dict_coordinates.items():
    df_prague.loc[df_prague.Neigborouhood == k,'latitude']=v[0]
    df_prague.loc[df_prague.Neigborouhood == k,'longitude']=v[1]
    
df_prague.head()

,ID,Neigborouhood,Geometry,Area,latitude,longitude
1,43,Praha 1,"[[[-743677.7200093269, -1043880.2599875033], [...",5538443.86,50.08728,14.41742
12,6,Praha 10,"[[[-735234.4999993183, -1045758.7199782543], [...",18599366.98,50.06055,14.55703
26,15,Praha 11,"[[[-734907.0599532239, -1049849.9600047208], [...",9793679.84,50.03178,14.50719
9,4,Praha 12,"[[[-741829.710013561, -1051335.8699600995], [-...",23317909.06,50.00564,14.40462
18,11,Praha 13,"[[[-750667.9699585326, -1047243.4200338311], [...",13196802.19,50.05163,14.34231


In [48]:
df_prague.head()

,ID,Neigborouhood,Geometry,Area,latitude,longitude
1,43,Praha 1,"[[[-743677.7200093269, -1043880.2599875033], [...",5538443.86,50.08728,14.41742
12,6,Praha 10,"[[[-735234.4999993183, -1045758.7199782543], [...",18599366.98,50.06055,14.55703
26,15,Praha 11,"[[[-734907.0599532239, -1049849.9600047208], [...",9793679.84,50.03178,14.50719
9,4,Praha 12,"[[[-741829.710013561, -1051335.8699600995], [-...",23317909.06,50.00564,14.40462
18,11,Praha 13,"[[[-750667.9699585326, -1047243.4200338311], [...",13196802.19,50.05163,14.34231


In [50]:
map_p = osm.graph_from_point((df_prague['latitude'].loc[0],df_prague['longitude'].loc[0]),distance=1000, network_type='walk')
osm.plot_graph(map_p)

AttributeError: module 'geopandas' has no attribute 'GeoDataFrame'